## DraftKings Points, Rebounds & Assist Lines w/ 5-Game Player Averages

### DraftKings

In [69]:
#Imports
import re
import urllib.request
from bs4 import BeautifulSoup
import json
from itertools import permutations
import numpy as np

#Read Data in From DraftKings
url = "https://sportsbook.draftkings.com/sites/US-NJ-SB/api/v1/eventgroup/103/full?format=json"
request = urllib.request.Request(url)
html = urllib.request.urlopen(request).read()

#Create JSON 
for line in data_lines:
    json_str = line.strip()
    json_dict = json.loads(json_str)
    
#Data About Points, Rebounds & Assists Subcategory of Player Props Stored Here
category = json_dict['eventGroup']['offerCategories'][1]['offerSubcategoryDescriptors'][3]['offerSubcategory']['offers']

#Number of Games for the Day That You Can Bet on. Make into a List to Loop Through
games = np.arange(0, len(category)).tolist()

#Create List for Number of Players
num_players = []

#Get Number of Players in Each Game and Append Player Counts to num_players list
for players in games:
    num_players.append(len(category[players]))
    
#Create a Range for the Number of Player Per Game
#For example - 7 Players in a Game would be [0,1,2,3,4,5,6]
num_players_range = []

#Create Range of Players for Each Game
for x in num_players:
    num_players_range.append(np.arange(0, x).tolist())

#Reset games and num_players_range to x and y     
x = games
y = num_players_range

#Create a list that holds the combination of games and players to loop through
#This will be used to get the player information by game from category
game_combo = [(i,j) for i in x for j in y[i]]

#List that will hold the player information dictionary 
player_info = []

#For each game and player combintation in game_combo append the player data to player_info
for x,y in game_combo:  
     player_info.append(category[x][y]['outcomes'])

#Create a list with the total number of players across both games
total_num_players = np.arange(0, len(player_info)).tolist()

#Create lists that will store data for each players lines and odds
participant = []
line = []
odds = []

#For each player in total_num_players, get the over and under (0,1) with the player's name, label, and oddsAmerican
for x in total_num_players:
    for y in (0,1):
        participant.append(player_info[x][y]['participant'])
        line.append(player_info[x][y]['label'])
        odds.append(player_info[x][y]['oddsAmerican'])

In [72]:
#Imports
import pandas as pd

#PRA = Points, Rebounds, Assists Table
PRA = pd.DataFrame({'PLAYER': participant,
                             'LINE': line,
                             'ODDS': odds})

#Create DataFrame
#Group by Player - Aggregate Since There are Two Rows For Each Player - One for Over and One for Under
PRA_Grouped = PRA.groupby('PLAYER').agg(lambda x: x.tolist())
PRA_Grouped

,LINE,ODDS
PLAYER,,
Bam Adebayo,"[Over 31.5, Under 31.5]","[-125, +103]"
Bradley Beal,"[Over 41.5, Under 41.5]","[-110, -110]"
Damian Lillard,"[Over 46.5, Under 46.5]","[-110, -110]"
Derrick Jones Jr.,"[Over 15.5, Under 15.5]","[-110, -110]"
Duncan Robinson,"[Over 16.5, Under 16.5]","[-104, -118]"
Enes Kanter,"[Over 29.5, Under 29.5]","[-110, -110]"
Gary Trent Jr.,"[Over 22.5, Under 22.5]","[-118, -104]"
Jimmy Butler,"[Over 36.5, Under 36.5]","[-104, -118]"
Kelly Olynyk,"[Over 15.5, Under 15.5]","[-104, -118]"


### Last Five Game Averages by Player

In [80]:
#Imports
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#There are Currently 5 Pages Worth of Stats on the Basketball.realgm.com Last 5 Games Averages Cagetory Section
#Create List (0,1,2,3,4,5) in case more pages are added. Will scrape page 6 (index 5) but nothing exists there
pages = np.arange(0,5)

#Create another list called rows. There are 100 rows per page. Each row represents a players stats. 
#For each page, we will need to get the data for each player by row
rows = np.arange(0,100)

#See 'iterations += 1' towards end of script for details
iterations = 0

#List that Collects Each Players Last 5 Game Stats
player_name = [] #Player Name
team_name = [] #Team the Player is on
mpg = [] #Minutes Per Game
fgm = [] #Field Goals Made
fga = [] #Field Goals Attempted
fgp = [] #Field Goal Percentage
threepm = [] #Three Pointers Made
threepa = [] #Three Pointers Attempted
threepp = [] #Three Point Percentage
ftm = [] #Free Throws Made
fta = [] #Free Throws Attempted
ftp = [] #Free Throw Percentage
rpg = [] #Rebounds Per Game
apg = [] #Assists Per Game
ppg = [] #Points Per Game

#For Each Page, do the Following 
for page in pages:
    
    #Get Request for Each Page - Note how str(page) will take each page from pages 
    response = get("https://basketball.realgm.com/nba/stats/2021/Averages/All/points/All/desc/"+ str(page)+"/Last_5_Games")
    
    #Sleep Random Number of Seconds Between 1 and 5 Before Sending Another Request to Avoid Being Blocked if too many Requests are Being Sent
    sleep(randint(1,5))
     
    #Throw Warning for Status Codes that are not 200
    #HTTP 200 OK Success Status Response Code Indicates that the Request has Succeeded
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #Define the HTML Text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #Define the Posts
    posts = page_html.find_all('tbody')
        
    #For Each Row Per Page, Get The Following Stats and Append to Respective List 
    #The script expects 100 rows of data, if there are less than the code will throw and IndexError
    #If IndexError occurs, the Script will Skip the Error until it Reaches 100 Rows Scraped and will move onto the Next Page
    for post in posts:
        for row in rows:
        
            #Player Name
            try:
                name = post.find_all('tr')[row].find_all('td')[1].text
                player_name.append(name)
            except IndexError:
                pass
                continue
                
            #Team
            try:
                team = post.find_all('tr')[row].find_all('td')[2].text
                team_name.append(team)
            except IndexError:
                pass
                continue
                
            #Minutes Per Game
            try:
                minutes_per_game = post.find_all('tr')[row].find_all('td')[4].text
                mpg.append(minutes_per_game)
            except IndexError:
                pass
                continue

            #Field Goals Made
            try:
                field_goals_made = post.find_all('tr')[row].find_all('td')[5].text
                fgm.append(field_goals_made)
            except IndexError:
                pass
                continue

            #Field Goals Attempts
            try:
                field_goals_attempts = post.find_all('tr')[row].find_all('td')[6].text
                fga.append(field_goals_attempts)
            except IndexError:
                pass
                continue
                
            #Field Goal %
            try:
                field_goals_percentage = post.find_all('tr')[row].find_all('td')[7].text
                fgp.append(field_goals_percentage)
            except IndexError:
                pass
                continue

            #3 Pointers Made
            try:
                three_pointers_made = post.find_all('tr')[row].find_all('td')[8].text
                threepm.append(three_pointers_made)
            except IndexError:
                pass
                continue
                
            #3 Pointers Attempted 
            try:
                three_pointers_attempted = post.find_all('tr')[row].find_all('td')[9].text
                threepa.append(three_pointers_attempted)
            except IndexError:
                pass
                continue
                
            #3 Point % 
            try:
                three_point_percentage = post.find_all('tr')[row].find_all('td')[10].text
                threepp.append(three_point_percentage)
            except IndexError:
                pass
                continue
            
            #Free Throws Made (FTM)
            try:
                free_throws_made = post.find_all('tr')[row].find_all('td')[11].text
                ftm.append(free_throws_made)
            except IndexError:
                pass
                continue
                
            #Free Throws Made (FTA)
            try:
                free_throws_attempted = post.find_all('tr')[row].find_all('td')[12].text
                fta.append(free_throws_attempted)
            except IndexError:
                pass
                continue
                
            #Free Throw % (FT%)
            try:
                free_throws_percentage = post.find_all('tr')[row].find_all('td')[13].text
                ftp.append(free_throws_percentage)
            except IndexError:
                pass
                continue

            #Rebounds Per Game (RPG)
            try:
                rebounds = post.find_all('tr')[row].find_all('td')[18].text
                rpg.append(rebounds)
            except IndexError:
                pass
                continue
                
            # #Assists Per Game (APG)
            try:
                assists = post.find_all('tr')[row].find_all('td')[19].text
                apg.append(assists)
            except IndexError:
                pass
                continue

            # #Points Per Game (PPG)
            try:
                points = post.find_all('tr')[row].find_all('td')[22].text
                ppg.append(points)
            except IndexError:
                pass
                continue
                
    #For each loop (page in pages) print 'Page X Scrape Successful'        
    iterations += 1
    print("Page " + str(iterations) + " Scrape Successful")

#Add space between 'Page x Scrape Successful' and "Basketball Stats Scraped Successfully"
print("\n")

#Once all pages in the search have been scraped, print "Basketball Stats Scraped Successfully"
print("Basketball Stats Scraped Successfully")

Page 1 Scrape Successful
Page 2 Scrape Successful
Page 3 Scrape Successful
Page 4 Scrape Successful
Page 5 Scrape Successful


Basketball Stats Scraped Successfully


In [17]:
#Imports
import pandas as pd

#Create Dataframe 
Five_Game_Average = pd.DataFrame({'PLAYER': player_name,
                            'TEAM':team_name,
                            'MPG': mpg,
                            'FGM': fgm,
                            'FGA': fga,
                            'FG%': fgp,
                            '3PM': threepm,
                            '3PA': threepa,
                            '3P%': threepp,
                            'FTM': ftm,
                            'FTA': fta,
                            'FT%': ftp,
                            'RPG': rpg,
                            'APG': apg,
                            'PPG': ppg})
#Print Dataframe
Five_Game_Average

,PLAYER,TEAM,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,RPG,APG,PPG
0,Luka Doncic,DAL,37.6,12.2,22.6,.540,4.0,8.4,.476,8.8,10.8,.815,8.0,9.4,37.2
1,Joel Embiid,PHI,35.6,11.6,22.0,.527,1.0,2.2,.455,11.0,12.8,.859,12.4,5.0,35.2
2,Zach LaVine,CHI,36.7,12.4,24.2,.512,4.6,9.2,.500,4.4,5.2,.846,7.0,4.6,33.8
3,Damian Lillard,POR,35.3,10.2,23.2,.440,4.2,11.6,.362,7.0,7.8,.897,3.8,10.6,31.6
4,Jamal Murray,DEN,36.7,11.6,20.0,.580,4.8,9.8,.490,3.4,3.8,.895,6.4,4.6,31.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,Ashton Hagans,MIN,2.0,0.0,0.0,.000,0.0,0.0,.000,0.0,0.0,.000,0.0,0.0,0.0
480,Jalen Harris,TOR,2.2,0.0,0.0,.000,0.0,0.0,.000,0.0,0.0,.000,0.5,0.0,0.0
481,Keljin Blevins,POR,4.3,0.0,0.6,.000,0.0,0.0,.000,0.0,0.0,.000,0.0,0.2,0.0
482,Noah Vonleh,BRK,2.7,0.0,0.8,.000,0.0,0.5,.000,0.0,0.0,.000,0.2,0.2,0.0


In [81]:
#Join Last 5 Game Player Averages with DraftKings Player Lines on Player Name
df_merge_col = pd.merge(PRA_Grouped, Five_Game_Average, on='PLAYER')

In [79]:
df_merge_col

,PLAYER,LINE,ODDS,TEAM,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,RPG,APG,PPG
0,Bam Adebayo,"[Over 31.5, Under 31.5]","[-125, +103]",MIA,35.1,7.6,13.6,.559,0.0,0.0,.000,3.0,3.8,.789,10.4,7.6,18.2
1,Bradley Beal,"[Over 41.5, Under 41.5]","[-110, -110]",WAS,35.6,10.0,21.2,.472,2.0,6.6,.303,9.2,9.8,.939,5.6,6.0,31.2
2,Damian Lillard,"[Over 46.5, Under 46.5]","[-110, -110]",POR,35.3,10.2,23.2,.440,4.2,11.6,.362,7.0,7.8,.897,3.8,10.6,31.6
3,Duncan Robinson,"[Over 16.5, Under 16.5]","[-104, -118]",MIA,32.6,5.0,11.0,.455,3.6,9.4,.383,0.2,0.2,1.000,3.2,1.2,13.8
4,Enes Kanter,"[Over 29.5, Under 29.5]","[-110, -110]",POR,29.1,5.4,10.0,.540,0.0,0.0,.000,2.0,2.8,.714,12.0,1.0,12.8
5,Jimmy Butler,"[Over 36.5, Under 36.5]","[-104, -118]",MIA,36.9,6.6,15.0,.440,0.6,1.8,.333,5.8,7.0,.829,10.0,9.8,19.6
6,Kelly Olynyk,"[Over 15.5, Under 15.5]","[-104, -118]",MIA,24.6,3.6,8.8,.409,0.6,4.6,.130,1.0,1.2,.833,7.2,1.4,8.8
7,Kendrick Nunn,"[Over 19.5, Under 19.5]","[-110, -110]",MIA,33.7,6.0,13.4,.448,3.0,8.0,.375,1.0,1.0,1.000,2.8,1.6,16.0
8,Kentavious Caldwell-Pope,"[Over 13.5, Under 13.5]","[-110, -110]",LAL,23.7,2.2,6.6,.333,0.8,3.6,.222,0.8,0.8,1.000,1.6,0.8,6.0
9,Kyle Kuzma,"[Over 24.5, Under 24.5]","[-110, -110]",LAL,31.7,6.6,11.8,.559,1.4,4.4,.318,0.8,1.2,.667,8.0,1.0,15.4
